In [1]:
import numpy as np
import random
import math
import time
import copy
import pandas as pd
import operator
from collections import defaultdict
np.set_printoptions(suppress=True)
import matplotlib.pyplot as plt

In [2]:
# Loading the single planar graph of 500 vertices
data = defaultdict(list)
for line in open("Graph500.txt"):
    split_line=line.split()
    ID_vertex = int(split_line[0])
    num_connected_vertices  = int(split_line[2])
    ID_connected_vertices = [int(i)-1 for i in split_line[3:]]
    if (ID_vertex) not in data.keys():
        data[ID_vertex].append(int(num_connected_vertices))
        data[ID_vertex].append(0)
        data[ID_vertex].append(0)
        data[ID_vertex].append(ID_connected_vertices)
        data[ID_vertex].append(0)
df = pd.DataFrame(data.values(),columns = ['Number of connected vertices','Gain', 'Fixed','ID connected vertices', 'Partition'])

num_vertices = len(df)
partition = random.sample(range(0,num_vertices),num_vertices//2)
df.loc[partition,'Partition'] = 1
df.head()

,Number of connected vertices,Gain,Fixed,ID connected vertices,Partition
0,8,0,0,"[27, 101, 161, 232, 359, 392, 459, 499]",1
1,2,0,0,"[78, 433]",1
2,3,0,0,"[164, 208, 256]",1
3,6,0,0,"[43, 201, 211, 278, 398, 440]",1
4,7,0,0,"[35, 92, 94, 145, 230, 303, 449]",0


In [3]:
class Node(object):
    def __init__(self, data = None, next_node = None, prev_node = None):
        self.data = data
        self.next_node =  next_node
        self.prev_node = prev_node

class DoublyLinkedList(object):
    def __init__(self, head = None):
        self.head = head
    
    def traverse(self):
        current_node = self.head
        while current_node != None:
            print(current_node.data)
            current_node = current_node.next_node
    
    def get_size(self):
        count = 0
        current_node = self.head
        while current_node != None:
            count += 1
            current_node = current_node.next_node
        return count
            
    def append(self, data):
        new_node = Node(data)
        current_node = self.head
        new_node.next_node = current_node
        new_node.prev_node = None
        if current_node != None:
            current_node.prev_node = new_node
        self.head = new_node
    
    def insert_end(self, data):
        new_node = Node(data)
        new_node.next = None
        if self.head == None:
            new_node.prev_node = None
            self.head = new_node
        return
    
        first_node = self.head
        while first_node.next_node:
            first_node = first_node.next_node
        first_node.next_node = new_node
        new_node.prev_node = first_node
    
    
    def delete(self, data):
        current_node = self.head
        while current_node != None:
            if current_node.data == data and current_node == self.head:
                if not current_node.next_node:
                    current_node = None
                    self.head = None
                    return
                else:
                    q = current_node.next_node
                    current_node.next_node = None
                    q.prev_node = None
                    current_node = None
                    self.head = q
                    return
            
            elif current_node.data == data:
                if current_node.next_node != None:
                    p = current_node.prev_node
                    q = current_node.next_node
                    p.next_node = q
                    q.prev_node = p
                    current_node.next_node = None
                    current_node.prev_node = None
                    current_node = None
                    return
                else:
                    p = current_node.prev_node
                    p.next_node = None
                    current_node.prev_node = None
                    current = None
                    return
            current_node =  current_node.next_node

class Bucket(object):
    def __init__(self, value):
        self.gain = value
        self.data = DoublyLinkedList()

class Bucket_Arrays(object):
    def __init__(self, maxdegree):
        self.bucket_range = maxdegree
        self.left_buckets = [Bucket(i) for i in range(-self.bucket_range,self.bucket_range+1)]
        self.right_buckets = [Bucket(i) for i in range(-self.bucket_range,self.bucket_range+1)]   

In [4]:
def calculate_num_cuts(df):
    connected_edges = df[df['Partition']==0]['ID connected vertices'].values
    outside_partition = list(df[df['Partition']==1].index)
    connected_edges = [item in outside_partition for sublist in connected_edges for item in sublist]
    num_cuts = np.sum(connected_edges)
    return num_cuts

def initialize_gain_buckets(df, buckets):
    max_degree = np.max(df['Number of connected vertices'])
    for i in range(len(df)):
        connected_edges = df['ID connected vertices'][i]
        partition_value = df['Partition'][i]
        current_partition = np.array(df[df['Partition']==partition_value].index)
        other_partition = np.array(df[df['Partition']==int(not(partition_value))].index)
        gain = len(np.intersect1d(connected_edges,other_partition))-len(np.intersect1d(connected_edges,current_partition))
        df.loc[i,'Gain'] = gain
        
        if partition_value == 0:
            buckets.left_buckets[gain+max_degree].data.append(i)
        else:
            buckets.right_buckets[gain+max_degree].data.append(i)
    return df,buckets

def calculate_vertex_max_gain(buckets):
    for bucket in buckets:
        max_gain_bucket = bucket.data.head
        if(max_gain_bucket != None):
            vertex_max_gain = max_gain_bucket.data
    return vertex_max_gain

def update_df_buckets(df,buckets,vertex_max_gain):
    max_degree = np.max(df['Number of connected vertices'])
    partition_value = df.loc[vertex_max_gain,'Partition']
    df.loc[vertex_max_gain,'Partition'] = int(not(partition_value))
    if df.loc[vertex_max_gain,'Partition'] == 1:
        gain = df.loc[vertex_max_gain,'Gain']
        buckets.left_buckets[gain+max_degree].data.delete(vertex_max_gain)
        df,buckets = re_calculate_gain(df, buckets,vertex_max_gain)
        
    else:
        gain = df.loc[vertex_max_gain,'Gain']
        buckets.right_buckets[gain+max_degree].data.delete(vertex_max_gain)
        df,buckets = re_calculate_gain(df, buckets,vertex_max_gain)
    return df, buckets

def re_calculate_gain(df, buckets, vertex_max_gain):
    max_degree = np.max(df['Number of connected vertices'])
    vertices_to_update = df.loc[vertex_max_gain]['ID connected vertices']
    
    for i in vertices_to_update:
        if(df.loc[i]['Fixed'] == 0):
            current_gain = df.loc[i]['Gain']
            current_partition = df.iloc[i]['Partition']
            if current_partition == 1:
                buckets.right_buckets[current_gain+max_degree].data.delete(i)
            else:
                buckets.left_buckets[current_gain+max_degree].data.delete(i)

            connected_edges = df['ID connected vertices'][i]
            partition_value = df['Partition'][i]
            current_partition = np.array(df[df['Partition']==partition_value].index)
            other_partition = np.array(df[df['Partition']==int(not(partition_value))].index)
            gain = len(np.intersect1d(connected_edges,other_partition))-len(np.intersect1d(connected_edges,current_partition))
            df.loc[i,'Gain'] = gain
            if(partition_value == 1): 
                buckets.right_buckets[gain+max_degree].data.append(i)
            else:
                buckets.left_buckets[gain+max_degree].data.append(i)
    df.loc[vertex_max_gain,'Fixed'] = 1
    df.loc[vertex_max_gain,'Gain'] = -999
    return df, buckets

In [5]:
def Fiduccia_Mattheyses_LS(df):
    start = time.time()
    num_cuts = calculate_num_cuts(df)
    min_cuts = num_cuts
    max_degree = np.max(df['Number of connected vertices'])
    buckets = Bucket_Arrays(max_degree)
    df,buckets = initialize_gain_buckets(df, buckets)
    save_partition = copy.deepcopy(df['Partition'].values)
    while(np.sum(df['Fixed']) < num_vertices):
        if(len(df[df['Partition']==0]) >= len(df[df['Partition']==1])):
            vertex_max_gain = calculate_vertex_max_gain(buckets.left_buckets)
            df, buckets = update_df_buckets(df,buckets,vertex_max_gain)
            
        else:
            vertex_max_gain = calculate_vertex_max_gain(buckets.right_buckets)
            df, buckets = update_df_buckets(df,buckets,vertex_max_gain)
            
        num_cuts = calculate_num_cuts(df)
        if(num_cuts < min_cuts and len(df[df['Partition']==0]) == len(df[df['Partition']==1])):
            save_partition = copy.deepcopy(df['Partition'].values)
            min_cuts = num_cuts
    end = time.time()
    print("Elapsed time ",end-start)
    print("Optimal cuts ", min_cuts)
    return save_partition

In [6]:
final_partition = Fiduccia_Mattheyses_LS(df)

Elapsed time  6.359986305236816
Optimal cuts  76
500


In [7]:
final_partition

array([1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0,

In [8]:
df.head()

,Number of connected vertices,Gain,Fixed,ID connected vertices,Partition
0,8,-999,1,"[27, 101, 161, 232, 359, 392, 459, 499]",0
1,2,-999,1,"[78, 433]",0
2,3,-999,1,"[164, 208, 256]",0
3,6,-999,1,"[43, 201, 211, 278, 398, 440]",0
4,7,-999,1,"[35, 92, 94, 145, 230, 303, 449]",1
